In [19]:
import run_info_utils

In [20]:
def get_partial_df(df, n_rows):
    if df.shape[0] < n_rows:
        return df
    else:
        return df[:n_rows]

In [21]:
df = run_info_utils.get_df_run_info()

In [22]:
df.head()

,all_correct_stop,batch_size,carry_list,confidence_prob,dev/carry-0/early_stopping/accuracy,dev/carry-1/early_stopping/accuracy,dev/carry-2/early_stopping/accuracy,dev/carry-3/early_stopping/accuracy,dev/carry-4/early_stopping/accuracy,dev/early_stopping/max_correct_answer_step,...,time/last_time,time/running_time,time/start_time,train_dev_test_ratio,train_set_size,train_set_size/carry-0,train_set_size/carry-1,train_set_size/carry-2,train_set_size/carry-3,train_set_size/carry-4
0,True,32,"[0, 1, 2, 3, 4]",0.9,1.000000,1.000000,1.000000,1.000000,1.00000,4.0,...,2019-03-25 21:09:13.650263,00:39:58.418681,2019-03-25 20:29:15.231582,"(1.0, 0, 0)",256,81.0,54.0,52.0,42.0,27.0
1,True,32,"[0, 1, 2, 3, 4]",0.9,1.000000,1.000000,1.000000,1.000000,1.00000,18.0,...,2019-03-25 20:36:27.179813,00:18:20.582719,2019-03-25 20:18:06.597094,"(1.0, 0, 0)",256,81.0,54.0,52.0,42.0,27.0
2,True,32,"[0, 1, 2, 3, 4]",0.9,1.000000,1.000000,1.000000,1.000000,1.00000,3.0,...,2019-03-26 08:44:56.883783,12:38:41.109794,2019-03-25 20:06:15.773989,"(1.0, 0, 0)",256,81.0,54.0,52.0,42.0,27.0
3,True,32,"[0, 1, 2, 3, 4]",0.9,0.901235,0.722222,0.673077,0.619048,0.37037,18.0,...,2019-03-25 17:04:05.709238,00:05:42.961018,2019-03-25 16:58:22.748220,"(1.0, 0, 0)",256,81.0,54.0,52.0,42.0,27.0
4,True,32,"[0, 1, 2, 3, 4]",0.9,1.000000,1.000000,1.000000,1.000000,1.00000,8.0,...,2019-03-25 20:39:46.803029,00:10:53.738680,2019-03-25 20:28:53.064349,"(1.0, 0, 0)",256,81.0,54.0,52.0,42.0,27.0


In [23]:
# print(list(df.columns))

In [24]:
experiment_name = 'iccm2019'
df = df.loc[df['experiment_name'] == experiment_name]

cols = ['run_id', 'operator', 'rnn_type', 'confidence_prob', 'operand_bits', 'hidden_activation', 'max_steps',
'dev/last_carry-0_mean_correct_answer_step', 
'dev/last_carry-1_mean_correct_answer_step', 
'dev/last_carry-2_mean_correct_answer_step', 
'dev/last_carry-3_mean_correct_answer_step', 
'dev/last_carry-4_mean_correct_answer_step', 
'dev/last_mean_correct_answer_step', 'dev/last_accuracy']
df = df[cols]

In [25]:
df = df.loc[df['dev/last_accuracy'] == 1.0]

In [26]:
df.shape

(300, 14)

In [27]:
df_add_jordan_9_relu_maxstep10 = df.query('operator == "add"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "relu"').query('max_steps == 10')
df_sub_jordan_9_relu_maxstep10 = df.query('operator == "subtract"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "relu"').query('max_steps == 10')
df_add_jordan_9_relu_maxstep20 = df.query('operator == "add"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "relu"').query('max_steps == 20')
df_sub_jordan_9_relu_maxstep20 = df.query('operator == "subtract"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "relu"').query('max_steps == 20')
df_add_jordan_9_relu_maxstep30 = df.query('operator == "add"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "relu"').query('max_steps == 30')
df_sub_jordan_9_relu_maxstep30 = df.query('operator == "subtract"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "relu"').query('max_steps == 30')
df_add_jordan_9_relu_maxstep40 = df.query('operator == "add"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "relu"').query('max_steps == 40')
df_sub_jordan_9_relu_maxstep40 = df.query('operator == "subtract"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "relu"').query('max_steps == 40')
df_add_jordan_9_relu_maxstep50 = df.query('operator == "add"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "relu"').query('max_steps == 50')
df_sub_jordan_9_relu_maxstep50 = df.query('operator == "subtract"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "relu"').query('max_steps == 50')
df_add_jordan_9_relu_maxstep60 = df.query('operator == "add"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "relu"').query('max_steps == 60')
df_sub_jordan_9_relu_maxstep60 = df.query('operator == "subtract"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "relu"').query('max_steps == 60')
df_add_jordan_9_relu_maxstep90 = df.query('operator == "add"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "relu"').query('max_steps == 90')


df_add_jordan_9_tanh_maxstep10 = df.query('operator == "add"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "tanh"').query('max_steps == 10')
df_sub_jordan_9_tanh_maxstep10 = df.query('operator == "subtract"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "tanh"').query('max_steps == 10')
df_add_jordan_9_tanh_maxstep20 = df.query('operator == "add"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "tanh"').query('max_steps == 20')
df_sub_jordan_9_tanh_maxstep20 = df.query('operator == "subtract"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "tanh"').query('max_steps == 20')
df_add_jordan_9_tanh_maxstep30 = df.query('operator == "add"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "tanh"').query('max_steps == 30')
df_sub_jordan_9_tanh_maxstep30 = df.query('operator == "subtract"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "tanh"').query('max_steps == 30')
df_add_jordan_9_tanh_maxstep40 = df.query('operator == "add"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "tanh"').query('max_steps == 40')
df_sub_jordan_9_tanh_maxstep40 = df.query('operator == "subtract"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "tanh"').query('max_steps == 40')
df_add_jordan_9_tanh_maxstep50 = df.query('operator == "add"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "tanh"').query('max_steps == 50')
df_sub_jordan_9_tanh_maxstep50 = df.query('operator == "subtract"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "tanh"').query('max_steps == 50')
df_add_jordan_9_tanh_maxstep60 = df.query('operator == "add"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "tanh"').query('max_steps == 60')
df_sub_jordan_9_tanh_maxstep60 = df.query('operator == "subtract"').query('rnn_type == "jordan"').query('confidence_prob == 0.9').query('hidden_activation == "tanh"').query('max_steps == 60')

In [28]:
print(df_add_jordan_9_relu_maxstep10.shape)
print(df_add_jordan_9_relu_maxstep20.shape)
print(df_add_jordan_9_relu_maxstep30.shape)
print(df_add_jordan_9_relu_maxstep40.shape)
print(df_add_jordan_9_relu_maxstep50.shape)
print(df_add_jordan_9_relu_maxstep60.shape)
print(df_add_jordan_9_relu_maxstep90.shape)

(0, 14)
(0, 14)
(300, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)


In [279]:
print(df_sub_jordan_9_relu_maxstep10.shape)
print(df_sub_jordan_9_relu_maxstep20.shape)
print(df_sub_jordan_9_relu_maxstep30.shape)
print(df_sub_jordan_9_relu_maxstep40.shape)
print(df_sub_jordan_9_relu_maxstep50.shape)
print(df_sub_jordan_9_relu_maxstep60.shape)

(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)


In [280]:
print(df_add_jordan_9_tanh_maxstep10.shape)
print(df_add_jordan_9_tanh_maxstep20.shape)
print(df_add_jordan_9_tanh_maxstep30.shape)
print(df_add_jordan_9_tanh_maxstep40.shape)
print(df_add_jordan_9_tanh_maxstep50.shape)
print(df_add_jordan_9_tanh_maxstep60.shape)

(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)


In [281]:
print(df_sub_jordan_9_tanh_maxstep10.shape)
print(df_sub_jordan_9_tanh_maxstep20.shape)
print(df_sub_jordan_9_tanh_maxstep30.shape)
print(df_sub_jordan_9_tanh_maxstep40.shape)
print(df_sub_jordan_9_tanh_maxstep50.shape)
print(df_sub_jordan_9_tanh_maxstep60.shape)

(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)
(0, 14)


In [199]:
n_rows = 240
df_add_jordan_9_relu_maxstep10 = get_partial_df(df_add_jordan_9_relu_maxstep10, n_rows)
print(df_add_jordan_9_relu_maxstep10.shape)
df_add_jordan_9_relu_maxstep20 = get_partial_df(df_add_jordan_9_relu_maxstep20, n_rows)
print(df_add_jordan_9_relu_maxstep20.shape)
df_add_jordan_9_relu_maxstep30 = get_partial_df(df_add_jordan_9_relu_maxstep30, n_rows)
print(df_add_jordan_9_relu_maxstep30.shape)
df_add_jordan_9_relu_maxstep40 = get_partial_df(df_add_jordan_9_relu_maxstep40, n_rows)
print(df_add_jordan_9_relu_maxstep40.shape)
df_add_jordan_9_relu_maxstep50 = get_partial_df(df_add_jordan_9_relu_maxstep50, n_rows)
print(df_add_jordan_9_relu_maxstep50.shape)

(0, 14)
(0, 14)
(210, 14)
(0, 14)
(0, 14)


# Export as CSV

## Functionalize

In [29]:
import pandas as pd
import numpy as np
import data_utils 
from utils import create_dir
from os.path import join

def get_csv_df(df, filename, experiment_name):
    # Get configurations
    n_rows = df.shape[0]
    operator = df['operator'].iloc[0]
    operand_digits = df['operand_bits'].iloc[0]
    carry_list = list(data_utils.import_carry_datasets(operand_digits, operator).keys())
    
    # Gather for each 
    csv_df_list = list()
    for carries in carry_list:
        col = 'dev/last_carry-{}_mean_correct_answer_step'.format(carries)
        csv_df = pd.DataFrame(data={'mean_anwer_steps':df[col], 'carries':np.full((n_rows), carries)})
        csv_df_list.append(csv_df)
    csv_df = pd.concat(csv_df_list, ignore_index=True)
    
    # Change the order of columns
    csv_df = csv_df[['mean_anwer_steps', 'carries']]
    
    # Create dir
    dir_to_save = join('result_statistics', experiment_name)
    create_dir(dir_to_save)
    
    # Save the dataframe to a CSV file.
    csv_df.to_csv(join(dir_to_save, filename), index=False)

In [30]:
get_csv_df(df_add_jordan_9_relu_maxstep30, 'df_add_jordan_9_relu_maxstep30.csv', experiment_name)

In [15]:
get_csv_df(df_add_jordan_9_relu_maxstep10, 'df_add_jordan_9_relu_maxstep10.csv', experiment_name)
get_csv_df(df_add_jordan_9_relu_maxstep20, 'df_add_jordan_9_relu_maxstep20.csv', experiment_name)
get_csv_df(df_add_jordan_9_relu_maxstep30, 'df_add_jordan_9_relu_maxstep30.csv', experiment_name)
get_csv_df(df_add_jordan_9_relu_maxstep40, 'df_add_jordan_9_relu_maxstep40.csv', experiment_name)
get_csv_df(df_add_jordan_9_relu_maxstep50, 'df_add_jordan_9_relu_maxstep50.csv', experiment_name)
get_csv_df(df_add_jordan_9_relu_maxstep60, 'df_add_jordan_9_relu_maxstep60.csv', experiment_name)

In [ ]:
get_csv_df(df_add_jordan_9_relu_maxstep10, 'df_add_jordan_9_relu_maxstep10.csv', experiment_name)
get_csv_df(df_sub_jordan_9_relu_maxstep10, 'df_sub_jordan_9_relu_maxstep10.csv', experiment_name)
get_csv_df(df_add_jordan_9_relu_maxstep20, 'df_add_jordan_9_relu_maxstep20.csv', experiment_name)
get_csv_df(df_sub_jordan_9_relu_maxstep20, 'df_sub_jordan_9_relu_maxstep20.csv', experiment_name)
get_csv_df(df_add_jordan_9_relu_maxstep30, 'df_add_jordan_9_relu_maxstep30.csv', experiment_name)
get_csv_df(df_sub_jordan_9_relu_maxstep30, 'df_sub_jordan_9_relu_maxstep30.csv', experiment_name)
get_csv_df(df_add_jordan_9_relu_maxstep40, 'df_add_jordan_9_relu_maxstep40.csv', experiment_name)
get_csv_df(df_sub_jordan_9_relu_maxstep40, 'df_sub_jordan_9_relu_maxstep40.csv', experiment_name)
get_csv_df(df_add_jordan_9_relu_maxstep50, 'df_add_jordan_9_relu_maxstep50.csv', experiment_name)
get_csv_df(df_sub_jordan_9_relu_maxstep50, 'df_sub_jordan_9_relu_maxstep50.csv', experiment_name)
get_csv_df(df_add_jordan_9_relu_maxstep60, 'df_add_jordan_9_relu_maxstep60.csv', experiment_name)
get_csv_df(df_sub_jordan_9_relu_maxstep60, 'df_sub_jordan_9_relu_maxstep60.csv', experiment_name)
get_csv_df(df_add_jordan_9_relu_maxstep90, 'df_add_jordan_9_relu_maxstep90.csv', experiment_name)

In [ ]:
get_csv_df(df_add_jordan_9_tanh_maxstep10, 'df_add_jordan_9_tanh_maxstep10.csv', experiment_name)
get_csv_df(df_sub_jordan_9_tanh_maxstep10, 'df_sub_jordan_9_tanh_maxstep10.csv', experiment_name)
get_csv_df(df_add_jordan_9_tanh_maxstep20, 'df_add_jordan_9_tanh_maxstep20.csv', experiment_name)
get_csv_df(df_sub_jordan_9_tanh_maxstep20, 'df_sub_jordan_9_tanh_maxstep20.csv', experiment_name)
get_csv_df(df_add_jordan_9_tanh_maxstep30, 'df_add_jordan_9_tanh_maxstep30.csv', experiment_name)
get_csv_df(df_sub_jordan_9_tanh_maxstep30, 'df_sub_jordan_9_tanh_maxstep30.csv', experiment_name)
get_csv_df(df_add_jordan_9_tanh_maxstep40, 'df_add_jordan_9_tanh_maxstep40.csv', experiment_name)
get_csv_df(df_sub_jordan_9_tanh_maxstep40, 'df_sub_jordan_9_tanh_maxstep40.csv', experiment_name)
get_csv_df(df_add_jordan_9_tanh_maxstep50, 'df_add_jordan_9_tanh_maxstep50.csv', experiment_name)
get_csv_df(df_sub_jordan_9_tanh_maxstep50, 'df_sub_jordan_9_tanh_maxstep50.csv', experiment_name)
get_csv_df(df_add_jordan_9_tanh_maxstep60, 'df_add_jordan_9_tanh_maxstep60.csv', experiment_name)
get_csv_df(df_sub_jordan_9_tanh_maxstep60, 'df_sub_jordan_9_tanh_maxstep60.csv', experiment_name)